# train_model

Import all the necessary packages

In [40]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
import seaborn as sns

#Machine learnin algo
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score, recall_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import Counter
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import scale
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

In [6]:
#To ignore warning
import warnings
warnings.filterwarnings('ignore')

Loading the data

In [7]:
data = pd.read_excel('INX_Future_Inc_Employee_Performance_CDS_Project2_Data_V1.8.xls',index=False)
data.head()

,EmpNumber,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,...,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,PerformanceRating
0,E1001000,32,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,10,3,...,4,10,2,2,10,7,0,8,No,3
1,E1001006,47,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,14,4,...,4,20,2,3,7,7,1,7,No,3
2,E1001007,40,Male,Life Sciences,Married,Sales,Sales Executive,Travel_Frequently,5,4,...,3,20,2,3,18,13,1,12,No,4
3,E1001009,41,Male,Human Resources,Divorced,Human Resources,Manager,Travel_Rarely,10,4,...,2,23,2,2,21,6,12,6,No,3
4,E1001010,60,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,16,4,...,4,10,1,3,2,2,2,2,No,3


Removal of Employee number which is not required.

In [8]:
data = data.drop(['EmpNumber'],axis=1)
data.head(2)

,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,EmpEnvironmentSatisfaction,...,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,PerformanceRating
0,32,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,10,3,4,...,4,10,2,2,10,7,0,8,No,3
1,47,Male,Marketing,Single,Sales,Sales Executive,Travel_Rarely,14,4,4,...,4,20,2,3,7,7,1,7,No,3


Label encoding the features to convert the string categorical variables to numerical data

In [9]:
processed_data = data
enc = LabelEncoder()
for i in (1,2,3,4,5,6,7,16,26):
    processed_data.iloc[:,i] = enc.fit_transform(data.iloc[:,i])
processed_data.head()

,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,EmpEnvironmentSatisfaction,...,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,PerformanceRating
0,32,1,2,2,5,13,2,9,3,4,...,4,10,2,2,10,7,0,8,No,1
1,47,1,2,2,5,13,2,13,4,4,...,4,20,2,3,7,7,1,7,No,1
2,40,1,1,1,5,13,1,4,4,4,...,3,20,2,3,18,13,1,12,No,2
3,41,1,0,0,3,8,2,9,4,2,...,2,23,2,2,21,6,12,6,No,1
4,60,1,2,2,5,13,2,15,4,1,...,4,10,1,3,2,2,2,2,No,1


In [10]:
#Saving the processed data as excel.
new_data = ExcelWriter('processed_data.xlsx')
processed_data.to_excel(new_data,'Sheet1')
new_data.save()

Defining X and Y - This step also includes backward elimination process of features.

In [106]:
X = data.loc[:,['EmpLastSalaryHikePercent','EmpEnvironmentSatisfaction','YearsSinceLastPromotion','EmpJobRole','EmpDepartment','EmpHourlyRate','ExperienceYearsInCurrentRole']]
y = data.PerformanceRating
X.head()


#- This step also includes backward elimination process of features.

#X = data.loc[:,['EmpLastSalaryHikePercent','EmpEnvironmentSatisfaction','YearsSinceLastPromotion']]
#X = data.loc[:,['EmpLastSalaryHikePercent','EmpEnvironmentSatisfaction','YearsSinceLastPromotion','EmpJobRole']]
#X = data.loc[:,['EmpLastSalaryHikePercent','EmpEnvironmentSatisfaction','YearsSinceLastPromotion','EmpJobRole','EmpDepartment']]
#X = data.loc[:,['EmpLastSalaryHikePercent','EmpEnvironmentSatisfaction','YearsSinceLastPromotion','EmpJobRole','EmpDepartment','EmpHourlyRate']]
#X = data.loc[:,['EmpLastSalaryHikePercent','EmpEnvironmentSatisfaction','YearsSinceLastPromotion','EmpJobRole','EmpDepartment','EmpHourlyRate','ExperienceYearsInCurrentRole','DistanceFromHome']]
#X = data.loc[:,['EmpLastSalaryHikePercent','EmpEnvironmentSatisfaction','YearsSinceLastPromotion','EmpJobRole','EmpDepartment','EmpHourlyRate','ExperienceYearsInCurrentRole','DistanceFromHome','TotalWorkExperienceInYears']]
#X = data.loc[:,['EmpLastSalaryHikePercent','EmpEnvironmentSatisfaction','YearsSinceLastPromotion','EmpJobRole','EmpDepartment','EmpHourlyRate','ExperienceYearsInCurrentRole','DistanceFromHome','TotalWorkExperienceInYears','EmpWorkLifeBalance']]


,EmpLastSalaryHikePercent,EmpEnvironmentSatisfaction,YearsSinceLastPromotion,EmpJobRole,EmpDepartment,EmpHourlyRate,ExperienceYearsInCurrentRole
0,1,4,0,13,5,55,7
1,1,4,1,13,5,42,7
2,10,4,1,13,5,48,13
3,4,2,12,8,3,73,6
4,3,1,2,13,5,84,2


Standardization technique is used - No change in accuracy even after standardization. Hence removed.

In [36]:
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

Train Test split

In [13]:
X_train, X_test, y_train, y_test = train_test_split (X,y, test_size = .3,random_state=10)

# Random Forest Classifier with Grid search

In [96]:
model_rf = RandomForestClassifier()  

# Hyperparameters tuning
parameters = {
        'n_estimators' : [100,200],
        'max_depth' : [4,5,6],
        'min_samples_split' : [1,2,3],
        'min_samples_leaf' : [7,6,5] ,
        'random_state': [0,1]
             }
grid = GridSearchCV(model_rf,parameters,cv=5)
grid.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [97]:
#To get best parameters
grid.best_params_

{'max_depth': 5,
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'n_estimators': 200,
 'random_state': 0}

In [98]:
# Predicting the model
y_testpredict_rf = grid.predict(X_test)
y_trainpredict_rf = grid.predict(X_train)
print('Best Parameters:',grid.best_params_)
print('Train data Accuracy score:', accuracy_score(y_train,y_trainpredict_rf))
print('Test data Accuracy score:', accuracy_score(y_test,y_testpredict_rf))
print('classification_report:')
print(classification_report(y_test,y_predict_rf))

Best Parameters: {'max_depth': 5, 'min_samples_leaf': 6, 'min_samples_split': 2, 'n_estimators': 200, 'random_state': 0}
Train data Accuracy score: 0.9345238095238095
Test data Accuracy score: 0.9222222222222223
classification_report:
              precision    recall  f1-score   support

           0       0.90      0.89      0.90        63
           1       0.94      0.97      0.95       264
           2       0.81      0.67      0.73        33

    accuracy                           0.93       360
   macro avg       0.89      0.84      0.86       360
weighted avg       0.92      0.93      0.92       360



In [39]:
print('Confustion Matrix:')
confusion_matrix(y_test,y_testpredict_rf)

Confustion Matrix:


array([[ 56,   7,   0],
       [  4, 255,   5],
       [  2,   9,  22]])

# K-Nearest Neighbours

In [18]:
model2 = KNeighborsClassifier(n_neighbors = 5)
model2.fit(X_train,y_train)
y_predictknn = model2.predict(X_test)
print(Counter(y_test))
print('Accuracy score:', accuracy_score(y_test,y_predictknn))
print('classification_report:')
print(classification_report(y_test,y_predictknn))

Counter({1: 264, 0: 63, 2: 33})
Accuracy score: 0.7361111111111112
classification_report:
              precision    recall  f1-score   support

           0       0.41      0.30      0.35        63
           1       0.80      0.89      0.85       264
           2       0.50      0.33      0.40        33

    accuracy                           0.74       360
   macro avg       0.57      0.51      0.53       360
weighted avg       0.71      0.74      0.72       360



In [19]:
print('Confustion Matrix:')
confusion_matrix(y_test,y_predictknn)

Confustion Matrix:


array([[ 19,  42,   2],
       [ 20, 235,   9],
       [  7,  15,  11]])

# Decision Tree

In [20]:
model3 = DecisionTreeClassifier(random_state = 10, criterion='entropy')
model3.fit(X_train, y_train)
y_predictdt = model3.predict(X_test)
print(Counter(y_test))
print('Accuracy score:', accuracy_score(y_test,y_predictdt))
print('classification_report:')
print(classification_report(y_test,y_predictdt))

Counter({1: 264, 0: 63, 2: 33})
Accuracy score: 0.8888888888888888
classification_report:
              precision    recall  f1-score   support

           0       0.84      0.83      0.83        63
           1       0.93      0.93      0.93       264
           2       0.68      0.70      0.69        33

    accuracy                           0.89       360
   macro avg       0.81      0.82      0.82       360
weighted avg       0.89      0.89      0.89       360



In [21]:
print('Confustion Matrix:')
confusion_matrix(y_test,y_predictdt)

Confustion Matrix:


array([[ 52,  11,   0],
       [  8, 245,  11],
       [  2,   8,  23]])

# Support Vector Machine

In [22]:
model4=SVC(C=8,gamma =0.53,kernel ='rbf')
model4.fit(X_train,y_train)
y_predictsvm = model4.predict(X_test)
print(Counter(y_test))
print('Accuracy score:', accuracy_score(y_test,y_predictsvm))
print('classification_report:')
print(classification_report(y_test,y_predictsvm))

Counter({1: 264, 0: 63, 2: 33})
Accuracy score: 0.7333333333333333
classification_report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.73      1.00      0.85       264
           2       0.00      0.00      0.00        33

    accuracy                           0.73       360
   macro avg       0.24      0.33      0.28       360
weighted avg       0.54      0.73      0.62       360



In [23]:
print('Confustion Matrix:')
confusion_matrix(y_test,y_predictsvm)

Confustion Matrix:


array([[  0,  63,   0],
       [  0, 264,   0],
       [  0,  33,   0]])

# XG Boost Classifer

In [65]:
model5 = XGBClassifier(learning_rate = 0.1, n_estimators = 40, random_state=3)
model5.fit(X_train,y_train)
y_testpredict = model5.predict(X_test)
y_trainpredict = model5.predict(X_train)
print(Counter(y_test))
print('Train data Accuracy score train:', accuracy_score(y_train,y_trainpredict))
print('Test data Accuracy score :', accuracy_score(y_test,y_testpredict))
print('classification_report:')
print(classification_report(y_test,y_testpredict))

Counter({1: 264, 0: 63, 2: 33})
Train data Accuracy score train: 0.9392857142857143
Test data Accuracy score : 0.9333333333333333
classification_report:
              precision    recall  f1-score   support

           0       0.90      0.89      0.90        63
           1       0.95      0.96      0.96       264
           2       0.82      0.82      0.82        33

    accuracy                           0.93       360
   macro avg       0.89      0.89      0.89       360
weighted avg       0.93      0.93      0.93       360



In [30]:
print('Confustion Matrix:')
confusion_matrix(y_test,y_testpredict)

Confustion Matrix:


array([[ 56,   7,   0],
       [  5, 253,   6],
       [  1,   5,  27]])

# Naive Bayes Bernoulli

In [31]:
from sklearn.naive_bayes import BernoulliNB
model_nb = BernoulliNB()
model_nb.fit(X_train,y_train)
y_predict_nb = model_nb.predict(X_test)
print('Accuracy score:', accuracy_score(y_test,y_predict_nb))
print('classification_report:')
print(classification_report(y_test,y_predict_nb))

Accuracy score: 0.7333333333333333
classification_report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.73      1.00      0.85       264
           2       0.00      0.00      0.00        33

    accuracy                           0.73       360
   macro avg       0.24      0.33      0.28       360
weighted avg       0.54      0.73      0.62       360



In [32]:
print('Confustion Matrix:')
confusion_matrix(y_test,y_predict_nb)

Confustion Matrix:


array([[  0,  63,   0],
       [  0, 264,   0],
       [  0,  33,   0]])

# Artificial Neural Network

In [33]:
model7 = MLPClassifier(hidden_layer_sizes=(200,100,100),batch_size=15,learning_rate_init=0.01,max_iter=2000,random_state=10)
model7.fit(X_train,y_train)
y_predictann = model7.predict(X_test)
print(Counter(y_test))
print('Accuracy score:', accuracy_score(y_test,y_predictann))
print('classification_report:')
print(classification_report(y_test,y_predictann))

Counter({1: 264, 0: 63, 2: 33})
Accuracy score: 0.8666666666666667
classification_report:
              precision    recall  f1-score   support

           0       0.83      0.62      0.71        63
           1       0.91      0.94      0.92       264
           2       0.64      0.76      0.69        33

    accuracy                           0.87       360
   macro avg       0.79      0.77      0.78       360
weighted avg       0.87      0.87      0.86       360



In [34]:
print('Confustion Matrix:')
confusion_matrix(y_test,y_predictann)

Confustion Matrix:


array([[ 39,  20,   4],
       [  6, 248,  10],
       [  2,   6,  25]])

# Exporting the model

In [103]:
#importing the required package
import joblib
#Taking in the Random Forest algorithm
joblib.dump(model_rf, 'prediction.ml')

['prediction.ml']